In [53]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  7 16:15:55 2022

@author: SkyMap
"""

import os, glob
import numpy as np
import pandas as pd
from datetime import datetime


from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
    
import xgboost as xgb
from scipy.ndimage import convolve
from keras.models import load_model

import numpy as np
import pandas as pd
from tqdm import tqdm

import rasterio
from rasterio.windows import Window
np.random.seed()

In [54]:
def write_window_many_chanel(output_ds, arr_c, s_h, e_h ,s_w, e_w, sw_w, sw_h, size_w_crop, size_h_crop):
    for c, arr in enumerate(arr_c):
        output_ds.write(arr[s_h:e_h,s_w:e_w],window = Window(sw_w, sw_h, size_w_crop, size_h_crop), indexes= c + 1)

In [55]:
def get_list_image_by_time(dir_img):
    list_name_file = os.listdir(dir_img)
    print(list_name_file)
    list_time = []
    for name in list_name_file:
        list_time.append(name[17:25])
    list_time.sort(key=lambda date: datetime.strptime(date, '%Y%m%d'))
    return list_time[:9]

In [56]:
def get_df_flatten_predict(img_window, list_number_band, name_atrr):
    dfObj = pd.DataFrame()
    i = 0
    for band in img_window:
        band = band.flatten()
        name_band = f"band {list_number_band[i]}_{name_atrr}"
        dfObj[name_band] = band
        i+=1
    return dfObj

In [57]:
def create_df_from_window(list_fp_img_large, list_name_file_sort, list_number_band, w_crop_start, h_crop_start, size_w_crop, size_h_crop):
    fp_large_first = [s for s in list_fp_img_large if list_name_file_sort[0] in s][0]
    with rasterio.open(fp_large_first) as src:
            img_window = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
    df_all_predict = get_df_flatten_predict(img_window, list_number_band, list_name_file_sort[0])

    for name_file_sort in list_name_file_sort[1:]:
        fp_large = [s for s in list_fp_img_large if name_file_sort in s][0]
        with rasterio.open(fp_large) as src:
            img_window = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
        df_one_img = get_df_flatten_predict(img_window, list_number_band, name_file_sort)
        df_all_predict = pd.concat([df_all_predict, df_one_img], axis=1)
    return df_all_predict, img_window.shape[1:]

In [58]:
def predict_df(df_all_predict, model, crop_size, shape_win):
    para = 4
    predictions = model.predict(df_all_predict, batch_size=1000, verbose=1)
    predictions = np.transpose(predictions)
    predictions = np.reshape(predictions, (-1, shape_win[0], shape_win[1]))
    # print(predictions)
    print('shape of predictions', predictions.shape)
    # kernel = [[1, 1, 1, 1, 1],
    #             [1, 1, 1, 1, 1],
    #             [1, 1, para, 1, 1],
    #             [1, 1, 1, 1, 1],
    #             [1, 1, 1, 1, 1]]

    # kernel = [[1, 1, 1],
    #         [1, para, 1],
    #         [1, 1, 1] ]
    # kernel = [1]
    # for i in range(len(predictions)):
    #     predictions[i] = convolve(predictions[i], kernel, mode='constant', cval=0.0)
    return np.array([np.argmax(predictions, axis=0).astype('uint16')])


In [59]:
# def predict_big(out_fp_predict, fp_img_large, input_size, crop_size, model, list_number_band):
def predict_big(out_fp_predict, list_fp_img_large, list_name_file_sort, list_number_band, crop_size, model):
    fp_large_first = [s for s in list_fp_img_large if list_name_file_sort[0] in s][0]
    with rasterio.open(fp_large_first) as src:
        h,w = src.height,src.width
        source_crs = src.crs
        source_transform = src.transform
        dtype_or = src.dtypes
        
    with rasterio.open(out_fp_predict, 'w', driver='GTiff',
                                height = h, width = w,
                                count=1, dtype=dtype_or[0],
                                crs=source_crs,
                                transform=source_transform,
                                nodata=0,
                                compress='lzw') as output_ds:
        output_ds = np.empty((1,h,w))
        
        
    input_size = crop_size    
    padding = int((input_size - crop_size)/2)
    list_weight = list(range(0, w, crop_size))
    list_hight = list(range(0, h, crop_size))

    with rasterio.open(out_fp_predict,"r+") as output_ds:
        with tqdm(total=len(list_hight)*len(list_weight)) as pbar:
            for start_h_org in list_hight:
                for start_w_org in list_weight:
                    # print('join')
                    # vi tri bat dau
                    h_crop_start = start_h_org - padding
                    w_crop_start = start_w_org - padding
                    # kich thuoc
                    
                    # truong hop 0 0
                    if h_crop_start < 0 and w_crop_start < 0:
                        # continue
                        h_crop_start = 0
                        w_crop_start = 0
                        size_h_crop = crop_size + padding
                        size_w_crop = crop_size + padding
                        
                        df_all_predict, shape_win = create_df_from_window(list_fp_img_large, list_name_file_sort, list_number_band, w_crop_start, h_crop_start, size_w_crop, size_h_crop)
                        img_predict = predict_df(df_all_predict, model, crop_size, shape_win) + 1
                        write_window_many_chanel(output_ds, img_predict, padding, crop_size + padding, padding, crop_size + padding, 
                                                                        start_w_org, start_h_org, crop_size, crop_size)
                    
                    # truong hop h = 0 va w != 0
                    elif h_crop_start < 0:
                        h_crop_start = 0
                        size_h_crop = crop_size + padding
                        size_w_crop = min(crop_size + 2*padding, w - start_w_org + padding)
                        if size_w_crop == w - start_w_org + padding:
                            end_c_index_w =  size_w_crop
                        else:
                            end_c_index_w = crop_size + padding

                        df_all_predict, shape_win = create_df_from_window(list_fp_img_large, list_name_file_sort, list_number_band, w_crop_start, h_crop_start, size_w_crop, size_h_crop)
                        img_predict = predict_df(df_all_predict, model, crop_size, shape_win) + 1
                        write_window_many_chanel(output_ds, img_predict, padding, crop_size + padding ,padding, end_c_index_w, 
                                                    start_w_org, start_h_org,  min(crop_size, w - start_w_org), crop_size)
                    
                    # Truong hop w = 0, h!=0 
                    elif w_crop_start < 0:
                        w_crop_start = 0
                        size_w_crop = crop_size + padding
                        size_h_crop = min(crop_size + 2*padding, h - start_h_org + padding)
                        
                        if size_h_crop == h - start_h_org + padding:
                            end_c_index_h =  size_h_crop
                        else:
                            end_c_index_h = crop_size + padding

                        df_all_predict, shape_win = create_df_from_window(list_fp_img_large, list_name_file_sort, list_number_band, w_crop_start, h_crop_start, size_w_crop, size_h_crop)
                        img_predict = predict_df(df_all_predict, model, crop_size, shape_win) + 1
                        write_window_many_chanel(output_ds, img_predict, padding, end_c_index_h, padding, crop_size + padding, 
                                                    start_w_org, start_h_org, crop_size, min(crop_size, h - start_h_org))
                        
                    # Truong hop ca 2 deu khac khong
                    else:
                        size_w_crop = min(crop_size +2*padding, w - start_w_org + padding)
                        size_h_crop = min(crop_size +2*padding, h - start_h_org + padding)
                        # img_window_crop  = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
                        # print(img_window_crop.shape, size_w_crop, size_h_crop)
                        if size_w_crop < (crop_size + 2*padding) and size_h_crop < (crop_size + 2*padding):
                            # print(img_window_crop.shape, size_w_crop, size_h_crop)
                            end_c_index_h = size_h_crop
                            end_c_index_w = size_w_crop
                            
                        elif size_w_crop < (crop_size + 2*padding):
                            end_c_index_h = crop_size + padding
                            end_c_index_w = size_w_crop
                            
                        elif size_h_crop < (crop_size + 2*padding):
                            end_c_index_w = crop_size + padding
                            end_c_index_h = size_h_crop
                            
                        else:
                            end_c_index_w = crop_size + padding
                            end_c_index_h = crop_size + padding
                            
                        df_all_predict, shape_win = create_df_from_window(list_fp_img_large, list_name_file_sort, list_number_band, w_crop_start, h_crop_start, size_w_crop, size_h_crop)
                        img_predict = predict_df(df_all_predict, model, crop_size, shape_win) + 1 
                        write_window_many_chanel(output_ds, img_predict, padding, end_c_index_h, padding, end_c_index_w, 
                                                    start_w_org, start_h_org, min(crop_size, w - start_w_org), min(crop_size, h - start_h_org))
                    pbar.update()
        output_ds.write_colormap(
                    1, {
                        1: (255, 0, 0, 255),
                        2: (255,255,0, 255),
                        3:(128,0,0,255),
                        4:(0,255,0,255),
                        5:(0,128,0,255),
                        6:(0,0,255,255) })

In [60]:
out_fp_predict = r"E:\WORK\Mongodia\Data\predict_time\run_with_nokerel_cut_box.tif"
# dir_img = r"E:\WORK\Mongodia\Data\Img"
# dir_img = r"E:\WORK\Mongodia\Data_cut_img\Img"
dir_img =r"E:\WORK\Mongodia\Data\Img_Cut_box"
list_number_band = [1,2,3,4,5,6,7]
crop_size = 1000

list_fp_img_large = glob.glob(os.path.join(dir_img, "*.tif"))
list_name_file_sort = get_list_image_by_time(dir_img)
print(list_name_file_sort)
model_fp = r"E:\WORK\Mongodia\Data_cut_img\modelDense.h5"
model = load_model(model_fp)
predict_big(out_fp_predict, list_fp_img_large, list_name_file_sort, list_number_band, crop_size, model)
print('done')
# img = rasterio.open(r"E:\WORK\Mongodia\Data\Img\LC08_L2SP_133026_20210201_20210302_02_T1.tif").read()
# img = rasterio.open(r"E:\WORK\Mongodia\Data_cut_img\Img\LC08_L2SP_133026_20210201_20210302_02_T1_0.tif").read()
img = rasterio.open(r"E:\WORK\Mongodia\Data\Img_Cut_box\LC08_L2SP_133026_20210201_20210302_02_T1_0.tif").read()
a= np.array([np.sum(img, axis=0)])
idx_nodata = np.where(a==0)

with rasterio.open(out_fp_predict, "r+") as dst:
    img = dst.read()
    img[idx_nodata] = 0
    dst.write(img)

['LC08_L2SP_133026_20210201_20210302_02_T1_0.tif', 'LC08_L2SP_133026_20210305_20210312_02_T1_0.tif', 'LC08_L2SP_133026_20210406_20210415_02_T1_0.tif', 'LC08_L2SP_133026_20210609_20210615_02_T1_0.tif', 'LC08_L2SP_133026_20210913_20210924_02_T1_0.tif', 'LC08_L2SP_133026_20211031_20211109_02_T1_0.tif', 'LC08_L2SP_133026_20220119_20220127_02_T1_0.tif', 'LC08_L2SP_133026_20220204_20220212_02_T1_0.tif', 'LC09_L2SP_133026_20211121_20220120_02_T1_0.tif', 'LC09_L2SP_133026_20211226_20220121_02_T1_0.tif', 'LC09_L2SP_133026_20220127_20220129_02_T1_0.tif', 'LC09_L2SP_133026_20220212_20220214_02_T1_0.tif', 'LC09_L2SP_133026_20220228_20220302_02_T1_0.tif']
['20210201', '20210305', '20210406', '20210609', '20210913', '20211031', '20211121', '20211226', '20220119']


  0%|          | 0/12 [00:00<?, ?it/s]

1000000/1000000 [==============================] - 5s 5us/step


  8%|▊         | 1/12 [00:07<01:26,  7.86s/it]

shape of predictions (6, 1000, 1000)
1000000/1000000 [==============================] - 4s 4us/step


...

 92%|█████████▏| 11/12 [00:58<00:04,  4.33s/it]

shape of predictions (6, 540, 1000)
71280/71280 [==============================] - 0s 4us/step


100%|██████████| 12/12 [00:59<00:00,  4.99s/it]

shape of predictions (6, 540, 132)


done
